# Naive Bayes Intent Classification

Super simple, naive bayes,

In [15]:
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

'en_US.UTF-8'

In [16]:
# setup
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.stats import norm
from sklearn import datasets
from sklearn.datasets.base import Bunch
from IPython.display import Image
import regex as re
%matplotlib inline

## Itsy Bitsy Teenie Weenie Silly Dataset
Dataset. Alt for få.

In [30]:
saldoNow = [
    u"Hvor mye penger har jeg på konto?",
    u"Hva er saldoen min i dag?",
    u"Hvor mye penger er det på kontoen min i dag?",
    u"saldo?",
    u"Hva er saldoen min nå?",
    u"Hva står på kontoen min nå?",
    u"Er det mye penger på kontoen min?",
    u"Hvor mange kroner har jeg?",
    u"Hvor mye penger har jeg?",
    u"Har jeg penger på konto?"
]

bruk = [
    u"Hvor mye penger brukte jeg i går?",
    u"Hvor mye penger brukte jeg på kiwi forrige uke?",
    u"Hva har jeg brukt på narvesen?",
    u"Bruker jeg mye penger på H&M?",
    u"Hvor mye penger bruker jeg i uka?",
    u"Hvor mange kroner har jeg sløst bort på kiosk?",
    u"Bruker jeg mye penger på klær?",
    u"Hvor mye penger bruker jeg i uka?",
    u"Hva har jeg brukt penger på i år?",
    u"Har jeg brukt penger på kebab i det siste?"
]

data = saldoNow + bruk
target = [1 for _ in range(len(saldoNow))] + [2 for _ in range(len(bruk))]
data, target

([u'Hvor mye penger har jeg p\xe5 konto?',
  u'Hva er saldoen min i dag?',
  u'Hvor mye penger er det p\xe5 kontoen min i dag?',
  u'saldo?',
  u'Hva er saldoen min n\xe5?',
  u'Hva st\xe5r p\xe5 kontoen min n\xe5?',
  u'Er det mye penger p\xe5 kontoen min?',
  u'Hvor mange kroner har jeg?',
  u'Hvor mye penger har jeg?',
  u'Har jeg penger p\xe5 konto?',
  u'Hvor mye penger brukte jeg i g\xe5r?',
  u'Hvor mye penger brukte jeg p\xe5 kiwi forrige uke?',
  u'Hva har jeg brukt p\xe5 narvesen?',
  u'Bruker jeg mye penger p\xe5 H&M?',
  u'Hvor mye penger bruker jeg i uka?',
  u'Hvor mange kroner har jeg sl\xf8st bort p\xe5 kiosk?',
  u'Bruker jeg mye penger p\xe5 kl\xe6r?',
  u'Hvor mye penger bruker jeg i uka?',
  u'Hva har jeg brukt penger p\xe5 i \xe5r?',
  u'Har jeg brukt penger p\xe5 kebab i det siste?'],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [31]:
from stop_words import get_stop_words
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("norwegian")
sw = get_stop_words('norwegian')

def clean(x):
    x = x.lower();
    x = re.sub(ur'[^\w\s]','',x,re.UNICODE) # bare bokstaver, ingen "?.,"
    x = x.split(" "); 
    x = filter(lambda w: w not in sw, x) # stop ord er ikke nyttig?
    x = map(stemmer.stem, x) # ordstammer, driter i bøyning av ord.
    x = " ".join(x);
    return x

In [32]:
data = map(clean, data)
data

[u'mye peng konto',
 u'saldo dag',
 u'mye peng konto dag',
 u'saldo',
 u'saldo',
 u'st\xe5r konto',
 u'mye peng konto',
 u'kron',
 u'mye peng',
 u'peng konto',
 u'mye peng brukt g\xe5r',
 u'mye peng brukt kiwi forr uke',
 u'brukt narves',
 u'bruk mye peng hm',
 u'mye peng bruk uka',
 u'kron sl\xf8st bort kiosk',
 u'bruk mye peng kl\xe6r',
 u'mye peng bruk uka',
 u'brukt peng \xe5r',
 u'brukt peng kebab sist']

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 
train_data_features = vectorizer.fit_transform(data)
train_data_features = train_data_features.toarray()

In [34]:
# Take a look at the words in the vocabulary
vocab = vectorizer.get_feature_names()
print vocab

[u'bort', u'bruk', u'brukt', u'dag', u'forr', u'g\xe5r', u'hm', u'kebab', u'kiosk', u'kiwi', u'kl\xe6r', u'konto', u'kron', u'mye', u'narves', u'peng', u'saldo', u'sist', u'sl\xf8st', u'st\xe5r', u'uka', u'uke', u'\xe5r']


In [35]:
import numpy as np

# Sum up the counts of each vocabulary word
dist = np.sum(train_data_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
for tag, count in zip(vocab, dist):
    print count, tag

1 bort
4 bruk
5 brukt
2 dag
1 forr
1 går
1 hm
1 kebab
1 kiosk
1 kiwi
1 klær
5 konto
2 kron
10 mye
1 narves
13 peng
3 saldo
1 sist
1 sløst
1 står
2 uka
1 uke
1 år


In [36]:
from sklearn.naive_bayes import BernoulliNB

cls = BernoulliNB()
cls.fit(train_data_features, target)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [27]:
def classify(x):
    return cls.predict(vectorizer.transform([clean(x)]).toarray())[0]

In [39]:
classify(u"hvor mye penger har jeg igjen?")

1

In [40]:
classify(u"hvor mye brukte jeg i går?")

2